# Prepaing the Dataset

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv('./dataset/glaucoma.csv')
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,OD,A,0
1,002.jpg,0.6953,OS,A,0
2,003.jpg,0.9629,OS,A,0
3,004.jpg,0.7246,OD,A,0
4,005.jpg,0.6138,OS,A,0


In [3]:
df.describe(include='all')

,Filename,ExpCDR,Eye,Set,Glaucoma
count,650,650.000000,650,650,650.000000
unique,650,NaN,2,2,NaN
top,001.jpg,NaN,OD,A,NaN
freq,1,NaN,327,325,NaN
mean,NaN,0.576308,NaN,NaN,0.258462
std,NaN,0.116395,NaN,NaN,0.438126
min,NaN,0.161000,NaN,NaN,0.000000
25%,NaN,0.496675,NaN,NaN,0.000000
50%,NaN,0.567650,NaN,NaN,0.000000
75%,NaN,0.648475,NaN,NaN,1.000000


In [4]:
print(df['Eye'].unique())
print(df['Set'].unique())
print(df['Glaucoma'].unique())

['OD' 'OS']
['A' 'B']
[0 1]


In [5]:
df['eye'] = df['Eye'].map({'OD': 0, 'OS': 1})
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye
0,001.jpg,0.7097,OD,A,0,0
1,002.jpg,0.6953,OS,A,0,1
2,003.jpg,0.9629,OS,A,0,1
3,004.jpg,0.7246,OD,A,0,0
4,005.jpg,0.6138,OS,A,0,1


In [6]:
df['set'] = df['Set'].map({'A': 0, 'B': 1})
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye,set
0,001.jpg,0.7097,OD,A,0,0,0
1,002.jpg,0.6953,OS,A,0,1,0
2,003.jpg,0.9629,OS,A,0,1,0
3,004.jpg,0.7246,OD,A,0,0,0
4,005.jpg,0.6138,OS,A,0,1,0


In [7]:
df['image'] = np.nan
df.head()

,Filename,ExpCDR,Eye,Set,Glaucoma,eye,set,image
0,001.jpg,0.7097,OD,A,0,0,0,NaN
1,002.jpg,0.6953,OS,A,0,1,0,NaN
2,003.jpg,0.9629,OS,A,0,1,0,NaN
3,004.jpg,0.7246,OD,A,0,0,0,NaN
4,005.jpg,0.6138,OS,A,0,1,0,NaN


In [8]:
image_dir = './dataset/Fundus_Scanes_Sorted'

In [9]:
train_dir = os.path.join(image_dir, 'Train')
validation_dir = os.path.join(image_dir, 'Validation')

In [10]:
mapping = {
    0: 'Glaucoma_Negative',
    1: 'Glaucoma_Positive'
}

In [11]:
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
train_image_negative = os.listdir(
    './dataset/Fundus_Scanes_Sorted/Train/Glaucoma_Negative/')
train_image_positive = os.listdir(
    './dataset/Fundus_Scanes_Sorted/Train/Glaucoma_Positive/')
print(len(train_image_negative) + len(train_image_positive))

520


In [13]:
test_image_negative = os.listdir(
    './dataset/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/')
test_image_positive = os.listdir(
    './dataset/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative/')
print(len(test_image_negative) + len(test_image_positive))

192


In [14]:
df_train = df[df['Filename'].isin(
    train_image_negative) | df['Filename'].isin(train_image_positive)]
len(df_train)

520

In [15]:
df_test = df[df['Filename'].isin(
    test_image_negative) | df['Filename'].isin(test_image_positive)]
len(df_test)

96

In [16]:
df_train['Glaucoma'].value_counts()

Glaucoma
0    386
1    134
Name: count, dtype: int64

In [17]:
IMAGE_SIZE = (224, 224)

In [18]:
def load_image(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(train_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [19]:
df_train['image'] = df_train.apply(load_image, axis=1)

/var/folders/9j/fmw5m6110hz9n8l4d20n37vr0000gn/T/ipykernel_82591/4091154328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['image'] = df_train.apply(load_image, axis=1)


In [20]:
def load_image1(row):
    global count
    folder_name = mapping[row['Glaucoma']]
    folder_path = os.path.join(validation_dir, folder_name)
    image_path = os.path.join(folder_path, row['Filename'])
    im = imread(image_path)
    im = resize(im, IMAGE_SIZE)
    return im

In [21]:
df_test['image'] = df_test.apply(load_image1, axis=1)

/var/folders/9j/fmw5m6110hz9n8l4d20n37vr0000gn/T/ipykernel_82591/4024071392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['image'] = df_test.apply(load_image1, axis=1)


In [22]:
X_image_train = df_train['image']
X_data_train = df_train[['ExpCDR', 'eye', 'set']]
y_train = df_train['Glaucoma']

In [23]:
X_image_test = df_test['image']
X_data_test = df_test[['ExpCDR', 'eye', 'set']]
y_test = df_test['Glaucoma']

In [24]:
X_image_train_stacked = np.stack(X_image_train.values)
X_image_test_stacked = np.stack(X_image_test.values)

In [25]:
from sklearn.utils.class_weight import compute_class_weight

In [26]:
class_weights = compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights

array([0.67357513, 1.94029851])

In [27]:
class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1]
}
class_weight_dict

{0: 0.6735751295336787, 1: 1.9402985074626866}

# Creating ML Model using Transfer Learning Approach 

In [28]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from sklearn.metrics import classification_report

In [29]:
resnet = ResNet50(input_shape=IMAGE_SIZE+(3,),
                  weights='imagenet', include_top=False)
vgg = VGG16(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
xception = Xception(input_shape=IMAGE_SIZE+(3,),
                    weights='imagenet', include_top=False)
resnet101 = ResNet101(input_shape=IMAGE_SIZE+(3,),
                      weights='imagenet', include_top=False)
inception = InceptionV3(input_shape=IMAGE_SIZE+(3,),
                        weights='imagenet', include_top=False)
mobilenet = MobileNetV2(input_shape=IMAGE_SIZE+(3,),
                        weights='imagenet', include_top=False)
efficientnet = EfficientNetB7(
    input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

In [30]:
model_dict = {
    'resnet50': resnet,
    'vgg16': vgg,
    'xception': xception,
    'resnet101': resnet101,
    'inception': inception,
    'mobilenet': mobilenet,
    'efficientnetB7': efficientnet
}
model_dict

{'resnet50': <keras.engine.functional.Functional at 0x359f05900>,
 'vgg16': <keras.engine.functional.Functional at 0x359f73310>,
 'xception': <keras.engine.functional.Functional at 0x3578cc100>,
 'resnet101': <keras.engine.functional.Functional at 0x35fa1b970>,
 'inception': <keras.engine.functional.Functional at 0x3608b2710>,
 'mobilenet': <keras.engine.functional.Functional at 0x360e56290>,
 'efficientnetB7': <keras.engine.functional.Functional at 0x364902e60>}

In [31]:
for key, model in model_dict.items():
    for layer in model.layers:
        layer.trainable = False

In [35]:
for key, model in model_dict.items():
    x = layers.Flatten()(model.output)
    prediction = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=model.input, outputs=prediction)
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    print(key)
    print(model.summary())

    # Training loop with dynamic stopping condition
    epochs = 0
    max_accuray = 0.85
    while True:
        history = model.fit(X_image_train_stacked, y_train, class_weight=class_weight_dict, epochs=1, verbose=1)
        epochs += 1
        if epochs >= 50 and history.history['accuracy'][0] > 0.95:
            break
        elif epochs > 100:
            if history.history['accuracy'][0] > max_accuray:
                break
            elif (epochs-100)%25 == 0:
                max_accuray -= 0.2

    # Evaluate model
    y_pred = model.predict(X_image_test_stacked)
    y_pred = [round(y[0]) for y in y_pred]
    print(classification_report(y_test, y_pred))

resnet50
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                   

 n)                                                                                               
                                                                                                  
 conv2_block1_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block1_1_relu[0][0]']    
                                                                                                  
 conv2_block1_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block1_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block1_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block1_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

3/3 [==============================] - 8s 3s/step
              precision    recall  f1-score   support

           0       1.00      0.65      0.78        96
           1       0.00      0.00      0.00         0

    accuracy                           0.65        96
   macro avg       0.50      0.32      0.39        96
weighted avg       1.00      0.65      0.78        96

xception
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_3[0][0]']                
   

/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

...

/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

 onv2D)                                                                                           
                                                                                                  
 block_16_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_16_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_16_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_16_depthwise_BN[0][0]']  
                                                                                                  
 block_16_project (Conv2D)      (None, 7, 7, 320)    307200      ['block_16_depthwise_relu[0][0]']
                                                                                                  
 block_16_project_BN (BatchNorm  (None, 7, 7, 320)   1280        ['block_16_project[0][0]']       
 alization

/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

                                                                                                  
 block2d_dwconv (DepthwiseConv2  (None, 56, 56, 288)  2592       ['block2d_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2d_bn (BatchNormalization  (None, 56, 56, 288)  1152       ['block2d_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block2d_activation (Activation  (None, 56, 56, 288)  0          ['block2d_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block2d_s

/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [36]:
data_input = layers.Input(shape=(3,))

In [37]:
x1 = layers.Dense(1, activation='relu')(data_input)
x1 = Model(inputs=data_input, outputs=x1)

In [38]:
x2 = layers.Flatten()(mobilenet.output)
x2 = Model(inputs=mobilenet.input, outputs=x2)

In [39]:
combined = layers.concatenate([x1.output, x2.output])

In [40]:
x_combined = layers.Dense(150, activation='relu')(combined)
x_combined = layers.Dense(50, activation='relu')(x_combined)
x_combined = layers.Dense(1, activation='sigmoid')(x_combined)

In [41]:
model = Model(inputs=[x1.input, x2.input], outputs=x_combined)

In [42]:
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                          

In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
epochs = 0
while True:
    history = model.fit([X_data_train, X_image_train_stacked], y=y_train,
                        class_weight=class_weight_dict, epochs=1, verbose=1)
    epochs += 1
    if epochs > 100 and history.history['accuracy'][0] > 0.95:
        break

17/17 [==============================] - 180s 11s/step - loss: 2.1500e-04 - accuracy: 1.0000


In [45]:
model.evaluate([X_data_test, X_image_test_stacked], y_test)

3/3 [==============================] - 2s 204ms/step - loss: 1.3500 - accuracy: 0.7812


[1.35004460811615, 0.78125]

In [46]:
y_pred = model.predict([X_data_test, X_image_test_stacked])
y_pred = [round(y[0]) for y in y_pred]

3/3 [==============================] - 1s 201ms/step


In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88        96
           1       0.00      0.00      0.00         0

    accuracy                           0.78        96
   macro avg       0.50      0.39      0.44        96
weighted avg       1.00      0.78      0.88        96



/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [48]:
x2_vgg = layers.Flatten()(vgg.output)
x2_vgg = Model(inputs=vgg.input, outputs=x2_vgg)

In [49]:
combined_vgg = layers.concatenate([x1.output, x2_vgg.output])

In [50]:
x_combined_vgg = layers.Dense(150, activation='relu')(combined_vgg)
x_combined_vgg = layers.Dense(50, activation='relu')(x_combined_vgg)
x_combined_vgg = layers.Dense(1, activation='sigmoid')(x_combined_vgg)

In [51]:
model_vgg = Model(inputs=[x1.input, x2_vgg.input], outputs=x_combined_vgg)

In [52]:
model_vgg.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [53]:
model_vgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
epochs = 0
while True:
    history = model_vgg.fit([X_data_train, X_image_train_stacked], y_train,
                            class_weight=class_weight_dict, epochs=1, verbose=1)
    epochs += 1
    if epochs > 100 and history.history['accuracy'][0] > 0.95:
        break

17/17 [==============================] - 101s 6s/step - loss: 0.0025 - accuracy: 1.0000


In [55]:
model_vgg.evaluate([X_data_test, X_image_test_stacked], y_test)

3/3 [==============================] - 8s 3s/step - loss: 0.0499 - accuracy: 0.9500


[0.04999781234, 0.95]

In [56]:
y_pred_vgg = model_vgg.predict([X_data_test, X_image_test_stacked])
y_pred_vgg = [round(y[0]) for y in y_pred_vgg]

3/3 [==============================] - 8s 3s/step


In [57]:
print(classification_report(y_test, y_pred_vgg))

              precision    recall  f1-score   support

           0       1.00      0.75      0.86        96
           1       0.00      0.00      0.00         0

    accuracy                           0.75        96
   macro avg       0.50      0.38      0.43        96
weighted avg       1.00      0.75      0.86        96



/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shamith/miniconda3/envs/glaucoma-detection/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

# Exporting the Model

In [58]:
model_vgg.save("./exports/glaucoma_detection_model_vgg_new.h5")